# Python pour l’analyse des cartes TEC IONEX 

Ce script Python a pour objectif de lire, traiter, décompresser, visualiser et analyser les cartes TEC (Total Electron Content) issues de fichiers IONEX. Ces cartes représentent la densité totale d’électrons dans l’ionosphère, une donnée importante pour la correction des signaux GNSS.

Ce travail s’inspire du notebook [github](https://github.com/daniestevez/jupyter_notebooks/blob/master/IONEX.ipynb).

In [ ]:
#télécharge les données IONEX

!git clone https://github.com/ClementineBeau/Fichiers-Atelier-IONEX.git

#Source explications fichier GIM https://gnss-x.ac.cn/docs/gimman.pdf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import matplotlib.colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import re                                                                
import cartopy.crs as ccrs                                                                               
import os
from unlzw3 import unlzw

In [ ]:
!pip install cartopy #executer si besoin
import cartopy

In [ ]:
!pip install unlzw3 #executer si besoin
import unlzw3

## Paramètres

In [ ]:
fig_size = [14, 10]
plt.rcParams['figure.figsize'] = fig_size
folder_ionex ="/content/Fichiers-Atelier-IONEX/data-ionex"   

## Fonctions

In [ ]:
def parse_map(tecmap, exponent=-1):
    '''
    Extrait une carte TEC depuis une chaîne de texte brute issue d’un fichier IONEX
    '''
    tecmap = re.split('.*END OF TEC MAP', tecmap)[0]
    return np.stack(
        [np.fromstring(l, sep=' ') for l in re.split('.*LAT/LON1/LON2/DLON/H\\n', tecmap)[1:]]) * 10 ** exponent

def get_tecmaps(filename):
    '''
    Lit un fichier IONEX et retourne toutes les cartes TEC qu’il contient
    '''
    with open(filename) as f:
        ionex = f.read()
        return [parse_map(t) for t in ionex.split('START OF TEC MAP')[1:]]

def get_tec(tecmap, lat, lon):
    #Extrait la valeur TEC à une latitude et longitude données.
    i = round((87.5 - lat) * (tecmap.shape[0] - 1) / (2 * 87.5))
    j = round((180 + lon) * (tecmap.shape[1] - 1) / 360)
    return tecmap[i, j]

def plot_tec_map(tecmap):
    proj = ccrs.PlateCarree()
    f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=proj))
    ax.coastlines()
    h = plt.imshow(tecmap, cmap='viridis', vmin=0, vmax=100, extent=(-180, 180, -87.5, 87.5), transform=proj)
    divider = make_axes_locatable(ax)
    ax_cb = divider.new_horizontal(size='5%', pad=0.1, axes_class=plt.Axes)
    f.add_axes(ax_cb)
    cb = plt.colorbar(h, cax=ax_cb)
    cb.set_label('TECU ($10^{16} \\mathrm{el}/\\mathrm{m}^2$)')

def get_tecmaps_with_time(filename):
    with open(filename) as f:
        ionex = f.read()
    # Séparer en blocs correspondant aux cartes
    blocs = ionex.split('START OF TEC MAP')[1:]
    cartes_dates = []
    for bloc in blocs:
        # Extraire la date juste avant "START OF TEC MAP" dans le texte complet
        date_match = re.search(r'(\d{4})\s+(\d{1,2})\s+(\d{1,2})\s+(\d{1,2})\s+(\d{1,2})\s+(\d{1,2})', bloc)
        if date_match:
            year, month, day, hour, minute, second = map(int, date_match.groups())
        else:
            year, month, day, hour, minute, second = (0, 0, 0, 0, 0, 0)

        tecmap = parse_map(bloc)
        cartes_dates.append(((year, month, day, hour, minute, second), tecmap))
    return cartes_dates

def decompress_ionex_file(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Fichier non trouvé : {filepath}")
    if filepath.lower().endswith(".z"):
        filepath_out = filepath[:-2]
    else:
        print(f"Fichier déjà décompressé: {filepath}")
        return filepath
    if os.path.exists(filepath_out):
        print(f"Fichier déjà décompressé : {filepath_out}")
        return filepath_out
    if filepath.lower().endswith(".z"):
        with open(filepath, 'rb') as f_in:
            data = unlzw(f_in.read())
        with open(filepath_out, 'wb') as f_out:
            f_out.write(data)
        print(f"Décompression .Z réussie, {filepath_out}")
    return filepath_out


def plot_tec_map_ax(tecmap, ax):
    ax.clear()
    ax.coastlines()
    proj = ccrs.PlateCarree()
    im = ax.imshow(tecmap, cmap='viridis', vmin=0, vmax=100,
                   extent=(-180, 180, -87.5, 87.5),
                   transform=proj)
    return im

## Tracer le TEC mondial - carte intéractive

In [ ]:
files_ionex_Z = sorted(os.listdir(folder_ionex))

if files_ionex_Z:
    first_file_path = os.path.join(folder_ionex, files_ionex_Z[0])
    file_ionex_i = decompress_ionex_file(first_file_path)
carte_dates = get_tecmaps_with_time(file_ionex_i)


proj = ccrs.PlateCarree()
fig, ax = plt.subplots(subplot_kw=dict(projection=proj), figsize=(14, 10))
plt.subplots_adjust(bottom=0.25)
date0, tecmap0 = carte_dates[0]
im = plot_tec_map_ax(tecmap0, ax)
title = ax.set_title(f"VTEC map — {date0[0]}-{date0[1]:02d}-{date0[2]:02d} {date0[3]:02d}:{date0[4]:02d}:{date0[5]:02d}")
cbar = fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('TECU ($10^{16} \\mathrm{el}/\\mathrm{m}^2$)')
ax_slider = plt.axes([0.2, 0.1, 0.6, 0.03])  # position [gauche, bas, largeur, hauteur]
slider = Slider(ax_slider, 'Temps', 0, len(carte_dates) - 1, valinit=0, valstep=1)
def update(val):
    idx = int(slider.val)
    date, tecmap = carte_dates[idx]
    im.set_data(tecmap)
    title.set_text(f"VTEC map — {date[0]}-{date[1]:02d}-{date[2]:02d} {date[3]:02d}:{date[4]:02d}:{date[5]:02d}")
    fig.canvas.draw_idle()
slider.on_changed(update)
plt.show()

## Tracer les variations journalières du TEC à Québec 

In [ ]:
lat_QC = 46.8139 #coordonnées hotel de ville de Québec
lon_QC = -71.2082
fig, ax = plt.subplots(figsize=(14, 6))

norm = matplotlib.colors.Normalize(vmin=1, vmax=8)
cmap = plt.cm.viridis
sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
lat_QC = 46.8139
lon_QC = -71.2082
for day_idx, file in enumerate(sorted(files_ionex_Z), start=1):
    file_path = os.path.join(folder_ionex, file)
    file_ionex_i = decompress_ionex_file(file_path)
    carte_dates = get_tecmaps_with_time(file_ionex_i)
    # Construction des paires (heure, tec) et tri
    data_points = [
        (date[3] + date[4] / 60.0, get_tec(tecmap, lat_QC, lon_QC))
        for date, tecmap in carte_dates
    ]
    data_points.sort()  # tri croissant sur l'heure
    heures, tec_values = zip(*data_points)  # unzip les listes triées
    ax.plot(heures, tec_values, color=cmap(norm(day_idx)), alpha=0.4)
ax.set_title('Variations journalières du VTEC à Québec (1 courbe par jour, 8 jours)')
ax.set_xlabel('Heure UTC')
ax.set_ylabel('TECU ($10^{16} \\mathrm{el}/\\mathrm{m}^2$)')
ax.grid(True)
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 25, 2))
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', label='Jour')
plt.tight_layout()
plt.show()
